porównanie

In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, rc
from scipy.stats import chisquare, kstest, norm, levene, ttest_ind, boxcox
from scipy.stats import probplot, kurtosis, skew, spearmanr, wilcoxon, chi2_contingency
from scipy.stats import mannwhitneyu, kruskal, pearsonr, ttest_rel, f_oneway
import pylab
import statsmodels.api as sm
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from matplotlib.colors import LogNorm, Normalize
from PIL import Image
import os
import joblib

In [ ]:
np.random.seed(seed=1)

funkcje z compare.ipynb

In [ ]:
def load_from_file(filename):
    file = open(filename, 'r')
    data = json.load(file)
    file.close()
    return data


def cube_list2array(d):
    d['cube'] = np.asarray(d['cube'])


def trim_cube(d, sh):
    if d['cube'].shape != sh:
        print("trimming")
        print("shape", d['cube'].shape)
        d['cube'] = d['cube'][:sh[0], :sh[1], :sh[2]]
        print("shape", d['cube'].shape)


def load_from_filename_list(fn_list):
    cube_list = []
    for fn in fn_list:
        cube = load_from_file(fn)
        cube_list2array(cube)
        trim_cube(cube, (180, 180, 240))
        cube_list.append(cube)
    return cube_list


def cube_by_name(all_cubes, name):
    for i in range(len(all_cubes)):
        if all_cubes[i]['name'] == name:
            return all_cubes[i].copy()
    return None


# próba kontrolna
def add_random_cube(all_cubes):
    if all_cubes[0]['name'] == 'random_cube':
        print('ta seria ma już random_cube!')
    else:
        c = all_cubes[0]['cube'] # my params
        sh0 = len(c)
        sh1 = len(c[0])
        sh2 = len(c[0][0])
        cube_shape = [sh0, sh1, sh2]
        cb = cube_shape

        random_arr = np.random.rand(cb[0], cb[1], cb[2])
        s = np.sum(random_arr)
        n_random_photons = 100_000_000
        random_arr *= n_random_photons
        random_arr /= s

        c2 = all_cubes[0]
        random_cube = {
            "n_photons": n_random_photons,
            "overflow": 0,
            "mu_a": 1.673,
            "name": 'random_cube',
            "bins_per_1_cm": c2['bins_per_1_cm'],
            "cube": random_arr,
            "photon_weight": 1.0,
            "normalized_already": False,
            "file_path": "NA",
            "file_dir_path": "NA",
            "params_type": "org_my"
        }

        print(random_arr.size)
        print(random_arr.shape)
        print(random_arr.sum())

        all_cubes.insert(0, random_cube)


def filter_outliers(arr, q_min=0.01, q_max=0.99):
    arr = arr.copy()
    if True:
        qqmin = np.quantile(arr, q=q_min)
        qqmax = np.quantile(arr, q=q_max)
        main_ids_out = np.logical_or( (arr < qqmin),  (arr > qqmax) )
        s = np.sum(main_ids_out)
        val_replace = np.random.choice(arr.flatten(), s)
        temp_arr = val_replace
        while True:
            ids_out = np.logical_or( (temp_arr < qqmin), (temp_arr > qqmax) )
            s = np.sum(ids_out)
            if s == 0:
                arr[main_ids_out] = temp_arr
                break
            val_replace = np.random.choice(arr.flatten(), s)
            temp_arr[ids_out] = val_replace
    return arr


def simple_hist_plot(arr, range, bins, title="", density=False, norm=False):
    # plt.hist(arr, range=range, bins=bins, density=density)
    # # plt.yscale("log") # - tutaj dopisać title i opisy osi
    # plt.show() #  - zakomentować to
    
    hist, bin_edges = np.histogram(arr, range=range, bins=bins, density=density)
    plt.bar(bin_edges[:-1], hist, width=(bin_edges[1] - bin_edges[0]) * 1.0) # , color='b'
    # plt.plot(bin_edges[:-1], hist, c='r')
    plt.title('Histogram'+title)
    plt.xlabel('transport')
    plt.ylabel('ilość próbek')
    if norm == "log":
        plt.yscale("log")
    plt.show()


def simple_boxplot(arr, title=""):
    plt.boxplot(arr)
    plt.title('Boxplot'+title)
    plt.show()


def filter_zeros(arr, typ=None):
    arr = arr.copy()
    arr[arr == 0] = np.min(arr[arr > 0])
    return arr


def fun_tan(arr, typ=None):
    return -np.tan(arr - np.pi / 2)


def fun_tan_wrapper(arr, typ): # łamana
    arr = arr.copy()
    global max_my
    global max_org
    global brpoint_my
    global brpoint_org

    if typ is None:
        raise ValueError('Potrzebny typ parametrów danych')
    
    if 'org' in typ:
        maxi_all = max_org
        brpoint = brpoint_org
    elif 'my' in typ:
        maxi_all = max_my
        brpoint = brpoint_my
    else:
        raise ValueError('Potrzebny typ parametrów danych')

    # dane od 0 do brpoint trzeba rozciągnąć na zakres 0 do pi/4
    # tan(pi/4) = 1
    arr = arr.copy()
    new_arr = arr.copy()
    new_arr[arr <= brpoint] = new_arr[arr <= brpoint] / brpoint * np.pi/4

    # dane od brpoint do maxi_all trzeba rozciągnąć na zakres od pi/4 do pi/2
    new_arr[arr > brpoint] = (new_arr[arr > brpoint] - brpoint) / (maxi_all-brpoint) * (np.pi/2 - np.pi/4) + np.pi/4

    arr = new_arr
    arr = fun_tan(arr)

    return arr


def move_mean(arr):
    arr = arr.copy()
    mea = np.mean(arr)
    arr = arr - mea
    return arr


def move_std(arr):
    arr = arr.copy()
    mea = np.mean(arr)
    st = np.std(arr)
    arr = arr - mea
    arr = arr / st
    arr = arr + mea
    return arr


def move_mean_std(arr):
    arr = arr.copy()
    mea = np.mean(arr)
    st = np.std(arr)
    arr = arr - mea
    arr = arr / st
    return arr


def dump_transformers():

    model_dir = 'models'
    files = os.listdir(model_dir)
    files = [model_dir+'/'+fn for fn in files]
    print(files)

    global org_quantile_transformer
    global org_power_transformer
    global org_global_training_arr
    global my_quantile_transformer
    global my_power_transformer
    global my_global_training_arr

    # Ścieżki do zapisania modeli
    org_quantile_transformer_filename = 'models/org_quantile_transformer.pkl'
    org_power_transformer_filename = 'models/org_power_transformer.pkl'
    org_global_training_arr_filename = 'models/org_global_training_arr.pkl'
    my_quantile_transformer_filename = 'models/my_quantile_transformer.pkl'
    my_power_transformer_filename = 'models/my_power_transformer.pkl'
    my_global_training_arr_filename = 'models/my_global_training_arr.pkl'

    # Zapisanie modeli za pomocą joblib
    if org_quantile_transformer is not None and (org_quantile_transformer_filename not in files):
        joblib.dump(org_quantile_transformer, org_quantile_transformer_filename)
    if org_power_transformer is not None and (org_power_transformer_filename not in files):
        joblib.dump(org_power_transformer, org_power_transformer_filename)
    if org_global_training_arr is not None and (org_global_training_arr_filename not in files):
        joblib.dump(org_global_training_arr, org_global_training_arr_filename)
    if my_quantile_transformer is not None and (my_quantile_transformer_filename not in files):
        joblib.dump(my_quantile_transformer, my_quantile_transformer_filename)
    if my_power_transformer is not None and (my_power_transformer_filename not in files):
        joblib.dump(my_power_transformer, my_power_transformer_filename)
    if my_global_training_arr is not None and (my_global_training_arr_filename not in files):
        joblib.dump(my_global_training_arr, my_global_training_arr_filename)


def load_transformers():

    model_dir = 'models'
    files = os.listdir(model_dir)
    files = [model_dir+'/'+fn for fn in files]
    print(files)

    loaded = 0

    global org_quantile_transformer
    global org_power_transformer
    global org_global_training_arr
    global my_quantile_transformer
    global my_power_transformer
    global my_global_training_arr

    # Ścieżki do zapisania modeli
    org_quantile_transformer_filename = 'models/org_quantile_transformer.pkl'
    org_power_transformer_filename = 'models/org_power_transformer.pkl'
    org_global_training_arr_filename = 'models/org_global_training_arr.pkl'
    my_quantile_transformer_filename = 'models/my_quantile_transformer.pkl'
    my_power_transformer_filename = 'models/my_power_transformer.pkl'
    my_global_training_arr_filename = 'models/my_global_training_arr.pkl'

    if org_quantile_transformer is None and (org_quantile_transformer_filename in files):
        org_quantile_transformer = joblib.load(org_quantile_transformer_filename)
        loaded += 1
    if org_power_transformer is None and (org_power_transformer_filename in files):
        org_power_transformer = joblib.load(org_power_transformer_filename)
        loaded += 1
    if org_global_training_arr is None and (org_global_training_arr_filename in files):
        org_global_training_arr = joblib.load(org_global_training_arr_filename)
        loaded += 1
    if my_quantile_transformer is None and (my_quantile_transformer_filename in files):
        my_quantile_transformer = joblib.load(my_quantile_transformer_filename)
        loaded += 1
    if my_power_transformer is None and (my_power_transformer_filename in files):
        my_power_transformer = joblib.load(my_power_transformer_filename)
        loaded += 1
    if my_global_training_arr is None and (my_global_training_arr_filename in files):
        my_global_training_arr = joblib.load(my_global_training_arr_filename)
        loaded += 1

    print(f'loaded num {loaded}')
    if loaded < 6:
        return False
    elif loaded == 6:
        return True
    else:
        raise NotImplementedError()


def train_data_transformers(init_arr = None):

    if load_transformers():
        return True

    global quantile_transformer
    global power_transformer
    global global_training_arr

    if quantile_transformer is None or power_transformer is None: # tutaj obsługowany
        # jest wyjątek braku ustawienia wcześniej transformerów globalnych
        if global_training_arr is None:
            if init_arr is None:
                raise Exception('Tablica treningowa nie została wcześniej ustawiona ani podana jako argument tej funkcji.')
            else:
                arr = init_arr.flatten()
                arr = np.sort(arr)
                arr = filter_outliers(arr)
                global_training_arr = arr
    else:
        print('Wszystkie transformery zostały przetrenowane wcześniej.')

    if quantile_transformer is None:
        print('Trenowanie quantile transformer.')
        rng = np.random.RandomState(304)
        # n_quantiles is set to the training set size rather than the default value
        # to avoid a warning being raised by this example
        n_quantiles = global_training_arr.shape[0]
        n_quantiles = 1_000_000
        subsample = n_quantiles
        qt = QuantileTransformer(
        n_quantiles=n_quantiles, output_distribution="normal", random_state=rng, subsample=subsample
        )
        quantile_transformer = qt.fit(global_training_arr.reshape(-1, 1))
        print('Training quantile transformer done.')

    if power_transformer is None:
        print('Trenowanie power transformer.')
        rng = np.random.RandomState(304)
        pt = PowerTransformer(method='yeo-johnson')
        power_transformer = pt.fit(global_training_arr.reshape(-1, 1))
        print('Training power transformer done.')


def setup_transformers():
    # wczytaj lub wytrenuj i zapisz

    global quantile_transformer
    global power_transformer
    global global_training_arr

    global org_quantile_transformer
    global org_power_transformer
    global org_global_training_arr
    global my_quantile_transformer
    global my_power_transformer
    global my_global_training_arr

    quantile_transformer = None
    power_transformer = None
    global_training_arr = None

    org_quantile_transformer = None
    org_power_transformer = None
    org_global_training_arr = None
    my_quantile_transformer = None
    my_power_transformer = None
    my_global_training_arr = None

    if load_transformers():
        return True

    train_arr = cube_by_name(all_cubes, 'mc456_my_100mln_cube')['cube']
    train_data_transformers(init_arr = train_arr)
    my_quantile_transformer = quantile_transformer
    my_power_transformer = power_transformer
    my_global_training_arr = global_training_arr
    quantile_transformer = None
    power_transformer = None
    global_training_arr = None
    print('my', my_quantile_transformer)
    print('empty', quantile_transformer)

    train_arr = cube_by_name(all_cubes, 'mc456_org_100mln_cube')['cube']
    train_data_transformers(init_arr = train_arr)
    org_quantile_transformer = quantile_transformer
    org_power_transformer = power_transformer
    org_global_training_arr = global_training_arr
    quantile_transformer = None
    power_transformer = None
    global_training_arr = None

    dump_transformers()


def ttest_preprocess(arr: np.ndarray, typ, dec=1):

    global org_power_transformer
    global my_power_transformer
    global org_quantile_transformer
    global my_quantile_transformer

    arr = arr.flatten()

    if dec == -1:
        return arr

    arr = filter_outliers(arr)

    if dec == 0:
        return arr

    if dec == 1:
        arr = filter_zeros(arr)
        arr = np.log(arr) # najlepszy boxcox lambda = 0, więc lepiej logarytm
        return arr

    if dec == 2:
        arr = fun_tan_wrapper(arr, typ) # przeskalowany logarytm
        return arr

    if dec == 3:
        if 'org' in cub['params_type']:
            transformer = org_power_transformer
        elif 'my' in cub['params_type']:
            transformer = my_power_transformer
        transformed_data = transformer.transform(arr.reshape(-1, 1)).reshape(-1)
        arr = transformed_data.flatten()
        return arr

    if dec == 4:
        if 'org' in cub['params_type']:
            transformer = org_quantile_transformer
        elif 'my' in cub['params_type']:
            transformer = org_quantile_transformer
        transformed_data = transformer.transform(arr.reshape(-1, 1)).reshape(-1)
        arr = transformed_data.flatten()
        return arr
    
    raise NotImplementedError('Nie ma takiego dec')

eksperymenty

In [ ]:
class experiment():
    def __init__(self, category, experiment_name, benchmark_path, folders):
        category = category #
        experiment_name = experiment_name #

        folders = folders #
        filenames = [] #
        benchmark_path = benchmark_path #
        mu_a = [] #
        all_cubes_names = [] #
        params_types = [] #

        cubes_path = 'CUBES'

        file_paths = []


    def get_all_cubes_names_from_filenames(self):
        for f_list in self.filenames:
            this_dir_cube_names = []
            for fn in f_list:
                cube_name = fn[:-5]
                this_dir_cube_names.append(cube_name)
            self.all_cubes_names.append(this_dir_cube_names)


    def get_filenames_from_folders(self):
        for f in self.folders:
            path = '/'.join(self.cubes_path, f)
            file_list = os.listdir(path)
            self.filenames.append(file_list)


    def mua_per_dir(self, mua_list):
        for mua, fn in zip(mua_list, self.filenames):
            self.mu_a.append([mua] * len(fn)) # łączenie tablic


    def params_types_per_dir(self, params_types):
        for pt, fn in zip(params_types, self.filenames):
            self.params_types.append([pt] * len(fn))


    def file_paths_from_filenames(self):
        for dir_id in range(len(self.folders)):
            this_folder_paths = []
            for fn_id in range(len(self.filenames[dir_id])):
                path = '/'.join(self.cubes_path, self.folders[dir_id], self.filenames[dir_id][fn_id])
                this_folder_paths.append(path)
            self.file_paths.append(this_folder_paths)


    def run(self):
        # load from files -> all_cubes
        all_cubes = []
        dir_num = len(self.folders)
        for dir_id in range(dir_num):
            path_list = self.file_paths[dir_id]
            all_cubes += load_from_filename_list(path_list)
        
        print('================================')
        print('len(all_cubes)', len(all_cubes))

        # przypisz właściwości podstawowe
        all_cubes_names = sum(self.all_cubes_names, []) # łączenie tablic
        all_mu_a = sum(self.mu_a, [])
        all_filename_list = sum(self.file_paths, [])
        all_dir_list = self.folders.copy()
        params_types = sum(self.params_types, [])
        for cub, name, mu_a, fn, fndir, param_t in zip(all_cubes, all_cubes_names, all_mu_a, all_filename_list, all_dir_list, params_types):
            cub['name'] = name
            cub['photon_weight'] = 1.0
            cub['normalized_already'] = False
            cub['mu_a'] = mu_a
            cub['file_path'] = fn
            cub['file_dir_path'] = fndir
            cub['params_type'] = param_t

        # jeśli random cube było wcześniej - wyrzucić je
        if all_cubes[0]['name'] == 'random_cube':
            all_cubes = all_cubes[1:]

        # dodaj losową próbę kontrolną
        add_random_cube(all_cubes)

        # USTAWIANIE WARTOŚCI SKRAJNYCH DO TRANSFORMACJI
        max_my = 55
        max_org = 105
        brpoint_my = 5
        brpoint_org = 0.015

        # print info
        print('================================')
        print("shape", all_cubes[0]['cube'].shape)
        print("size", all_cubes[0]['cube'].size)
        print("len", len(all_cubes))













        quantile_transformer = None
        power_transformer = None
        global_training_arr = None

        org_quantile_transformer = None
        org_power_transformer = None
        org_global_training_arr = None
        my_quantile_transformer = None
        my_power_transformer = None
        my_global_training_arr = None




python_vs_c = experiment(category = 'porównanie python - język c',
                         experiment_name = 'mc456 my_params 10k-100mln'+'_vs_'+'mc456_p my-params 10k-10mln',
                         benchmark_path = 'CUBES/mc456 my_params 10k-100mln/mc456_mc_100mln_my_params_cube.json',
                         folders = ['mc456 my_params 10k-100mln', 'mc456_p my-params 10k-10mln'],
                         )
python_vs_c.get_filenames_from_folders()
python_vs_c.get_all_cubes_names_from_filenames()
python_vs_c.mua_per_dir([1.673, 1.673])
python_vs_c.mua_per_dir(['my', 'my'])











PORÓWNYWANIE OGÓLNE:

mati-sim my_params 10k-100mln
mc456 my_params 10k-100mln

mati-sim org_params 10k-10mln
mc456 org_params 10k-100mln



specjalistyczne:

mc456 rozne g (0.0-1.0 z krokiem 0.1) 100mln

mati-sim my_params 10k g {0, 0.5, 0.9, 1}

mc456 rozne_skóry_z_tabeli (8 rodzajów) 100mln

mc456 my-params light-sources



dodatkowe:

mati-sim 2-layers

mati-sim 3-layers

mati-sim veins


In [3]:
os.listdir('CUBES')

['mati-sim 2-layers',
 'mati-sim 3-layers',
 'mati-sim my_params 10k g {0, 0.5, 0.9, 1}',
 'mati-sim my_params 10k-100mln',
 'mati-sim org_params 10k-10mln',
 'mati-sim veins',
 'mc456 my-params light-sources',
 'mc456 my_params 10k-100mln',
 'mc456 org_params 10k-100mln',
 'mc456 rozne g (0.0-1.0 z krokiem 0.1) 100mln',
 'mc456 rozne_skóry_z_tabeli (8 rodzajów) 100mln',
 'mc456_p my-params 10k-10mln']

In [4]:
[123]*4

[123, 123, 123, 123]

In [ ]:
porównanie python - język c:

mc456 my_params 10k-100mln
mc456_p my-params 10k-10mln




PORÓWNYWANIE OGÓLNE:

mati-sim my_params 10k-100mln
mc456 my_params 10k-100mln

mati-sim org_params 10k-10mln
mc456 org_params 10k-100mln



specjalistyczne:

mc456 rozne g (0.0-1.0 z krokiem 0.1) 100mln

mati-sim my_params 10k g {0, 0.5, 0.9, 1}

mc456 rozne_skóry_z_tabeli (8 rodzajów) 100mln

mc456 my-params light-sources



dodatkowe:

mati-sim 2-layers

mati-sim 3-layers

mati-sim veins